In [26]:
from sympy.parsing.sympy_parser import parse_expr
from sympy import *

In [27]:
def func(var1 = 'x',var2 = 'y',f = 'x**2 - x*y + y**2 + 9*x - 6 * y + 20'):
    return f.replace(var1,'x[0]').replace(var2,'x[1]')
func()

'x[0]**2 - x[0]*x[1] + x[1]**2 + 9*x[0] - 6 * x[1] + 20'

In [28]:
def gradFunc(var1 = 'x',var2 = 'y', f = 'x**2 - x*y + y**2 + 9*x - 6 * y + 20' ):
    x = Symbol(var1)
    y = Symbol(var2)
    f = parse_expr(f)
    return str(f.diff(x)).replace('x','x[0]').replace('y','x[1]') , str(f.diff(y)).replace('x','x[0]').replace('y','x[1]')
gradFunc()

('2*x[0] - x[1] + 9', '-x[0] + 2*x[1] - 6')

In [29]:
def f(x):
    return x[0]**2 - x[0]*x[1] + x[1]**2 + 9*x[0] - 6*x[1] + 20

# gradient
def f1(x):
    return np.array([2 * x[0] - x[1] + 9, -x[0] + 2*x[1] - 6])

In [30]:
import numpy.linalg as ln
import scipy
import numpy as np
import pandas as pd
import scipy.optimize
import math
import time

In [71]:

def BFGS(func = f,grad = f1, x0 =[1,1] , v1 = 10**(-4), v2 = 0.1,
         xmax =100 , interv = 10 **(-8), maxiter = 500, p1 = False, p2 = False ):
    start_time = time.time()
    k = 0
    gfk = grad(x0)
    I = np.eye(2, dtype=int)
    Hk = I
    xk = x0
    XK1 = []
    XK2 = []
    K = []
    F = []
    while ln.norm(gfk) > v1 and k < maxiter:
        
        k += 1
        pk = -np.dot(Hk, gfk)

        line_search = scipy.optimize.line_search(func, grad, xk, pk, c1 = v1, c2 = v2)
        alpha_k = line_search[0]
        if alpha_k is not None:
            
            xkp1 = xk + alpha_k * pk
            sk = xkp1 - xk
            xk = xkp1
        
            gfkp1 = grad(xkp1)
            yk = gfkp1 - gfk
            gfk = gfkp1
        
            ro = 1.0 / (np.dot(yk, sk))
            A1 = I - ro * sk[:, np.newaxis] * yk[np.newaxis, :]
            A2 = I - ro * yk[:, np.newaxis] * sk[np.newaxis, :]
            Hk = np.dot(A1, np.dot(Hk, A2)) + (ro * sk[:, np.newaxis] *
                                                 sk[np.newaxis, :])
            if p1:
                print(xk, func(xk))
       
            if p2 :
                XK1.append(xk[0])
                XK2.append(xk[1])
                K.append(k)
                F.append(func(xk))
        else:
            print('Выполнено с ошибкой, альфа не может быть найдена с такими условиями')
            print("время выполнения( в секундах ) = %s " % (time.time() - start_time))
            print()
            return 4
            break

    if p2 :
        data = pd.DataFrame(np.array([XK1,XK2,K,F]).transpose(), columns = ['x1', 'x2','k','F'])
        print(data)
        
    if k == maxiter :
        print (f'Координаты точки экстремума : {xk}')
        print (f'Значение функции в точке экстремума : {func(xk)}')
        print('Макс итераций')
        print("время выполнения( в секундах ) = %s " % (time.time() - start_time))
        print()
        return (xk,func(xk),2)
    elif any(xk) > xmax :
        print (f'Координаты точки экстремума : {xk}')
        print (f'Значение функции в точке экстремума : {func(xk)}')
        print('Ограничение на макс возможное значение аргумента')
        print("время выполнения( в секундах ) = %s " % (time.time() - start_time))
        print()
        return (xk,func(xk),3)
    else :
        print (f'Координаты точки экстремума : {xk}')
        print (f'Значение функции в точке экстремума : {func(xk)}')
        print('Точка, удовлетворяющая условиям найдена')
        print(f'время выполнения( в секундах ) = {(time.time() - start_time)}')
        print()
        return  (xk,func(xk), 1)
    

In [32]:
BFGS(f, f1)

Координаты точки экстремума : [-4.  1.]
Значение функции в точке экстремума : -1.0
Точка, удовлетворяющая условиям найдена
время выполнения( в секундах ) = 0.000997304916381836



(array([-4.,  1.]), -1.0, 1)

In [33]:
BFGS(f, f1,p1 = True)

[-2.57142857  2.78571429] 1.6785714285714235
[-4.10204082  0.87244898] -0.986333819241981
[-4.  1.] -1.0
Координаты точки экстремума : [-4.  1.]
Значение функции в точке экстремума : -1.0
Точка, удовлетворяющая условиям найдена
время выполнения( в секундах ) = 0.0009975433349609375



(array([-4.,  1.]), -1.0, 1)

In [34]:
BFGS(f, f1,p2 = True)

         x1        x2    k         F
0 -2.571429  2.785714  1.0  1.678571
1 -4.102041  0.872449  2.0 -0.986334
2 -4.000000  1.000000  3.0 -1.000000
Координаты точки экстремума : [-4.  1.]
Значение функции в точке экстремума : -1.0
Точка, удовлетворяющая условиям найдена
время выполнения( в секундах ) = 0.0039539337158203125



(array([-4.,  1.]), -1.0, 1)

### 1)

In [35]:
def func(var1 = 'x',var2 = 'y',f = 'x/(x**2 + y)'):
    return f.replace(var1,'x[0]').replace(var2,'x[1]')
func()

'x[0]/(x[0]**2 + x[1])'

In [36]:
def gradFunc(var1 = 'x',var2 = 'y', f = 'x/(x**2 + y)'):
    x = Symbol(var1)
    y = Symbol(var2)
    f = parse_expr(f)
    return str(f.diff(x)).replace('x','x[0]').replace('y','x[1]') , str(f.diff(y)).replace('x','x[0]').replace('y','x[1]')
gradFunc()

('-2*x[0]**2/(x[0]**2 + x[1])**2 + 1/(x[0]**2 + x[1])',
 '-x[0]/(x[0]**2 + x[1])**2')

In [37]:
def f(x):
    return x[0]**2 - x[0]*x[1] + x[1]**2 + 9*x[0] - 6*x[1] + 20

# gradient
def f1(x):
    return np.array([-2*x[0]**2/(x[0]**2 + x[1])**2 + 1/(x[0]**2 + x[1]), -x[0]/(x[0]**2 + x[1])**2])

In [38]:
BFGS(f, f1)

Выполнено с ошибкой, альфа не может быть найдена с такими условиями
время выполнения( в секундах ) = 0.0009963512420654297 



D:\Anaconda\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


4

### 2)

In [39]:
def func(var1 = 'x',var2 = 'y',f = '(x+y)**5 - 2 * (x+y)**4'):
    return f.replace(var1,'x[0]').replace(var2,'x[1]')
func()

'(x[0]+x[1])**5 - 2 * (x[0]+x[1])**4'

In [40]:
a = parse_expr('(x+y)**5 - 2 * (x+y)**4')

In [41]:
var1 = 'x'
var2 = 'y'

In [42]:
x = Symbol(var1)
y = Symbol(var2)

In [43]:
str(a.diff(x)).replace('x','x[0]').replace('y','x[1]')

'5*(x[0] + x[1])**4 - 8*(x[0] + x[1])**3'

In [44]:
str(a.diff(y)).replace('x','x[0]').replace('y','x[1]')

'5*(x[0] + x[1])**4 - 8*(x[0] + x[1])**3'

In [65]:
def f(x):
    return (x[0]+x[1])**5 - 2 * (x[0]+x[1])**4

# gradient
def f1(x):
    return np.array([5*(x[0] + x[1])**4 - 8*(x[0] + x[1])**3, 5*(x[0] + x[1])**4 - 8*(x[0] + x[1])**3])

In [73]:
BFGS(f, f1, x0 =[1,0])

Координаты точки экстремума : [1.29999973 0.29999973]
Значение функции в точке экстремума : -2.621439999996918
Точка, удовлетворяющая условиям найдена
время выполнения( в секундах ) = 0.0019664764404296875



(array([1.29999973, 0.29999973]), -2.621439999996918, 1)

### 3)

In [48]:
def func(var1 = 'x',var2 = 'y',f = 'x**3 + x*y**2 + x**2 + y**2'):
    return f.replace(var1,'x[0]').replace(var2,'x[1]')
func()

'x[0]**3 + x[0]*x[1]**2 + x[0]**2 + x[1]**2'

In [49]:
def gradFunc(var1 = 'x',var2 = 'y', f = 'x**3 + x*y**2 + x**2 + y**2'):
    x = Symbol(var1)
    y = Symbol(var2)
    f = parse_expr(f)
    return str(f.diff(x)).replace('x','x[0]').replace('y','x[1]') , str(f.diff(y)).replace('x','x[0]').replace('y','x[1]')
gradFunc()

('3*x[0]**2 + 2*x[0] + x[1]**2', '2*x[0]*x[1] + 2*x[1]')

In [54]:
def f(x):
    return x[0]**3 + x[0]*x[1]**2 + x[0]**2 + x[1]**2

# gradient
def f1(x):
    return np.array([3*x[0]**2 + 2*x[0] + x[1]**2, 2*x[0]*x[1] + 2*x[1]])

In [74]:
BFGS(f, f1, x0 =[1,0])

Координаты точки экстремума : [1.29999973 0.29999973]
Значение функции в точке экстремума : -2.621439999996918
Точка, удовлетворяющая условиям найдена
время выполнения( в секундах ) = 0.0019958019256591797



(array([1.29999973, 0.29999973]), -2.621439999996918, 1)

### 4)

In [61]:
def func(var1 = 'x',var2 = 'y',f = 'x**3 + 3*x*y**2 - 15*x -12 * y'):
    return f.replace(var1,'x[0]').replace(var2,'x[1]')
func()

'x[0]**3 + 3*x[0]*x[1]**2 - 15*x[0] -12 * x[1]'

In [62]:
def gradFunc(var1 = 'x',var2 = 'y', f = 'x**3 + 3*x*y**2 - 15*x -12 * y'):
    x = Symbol(var1)
    y = Symbol(var2)
    f = parse_expr(f)
    return str(f.diff(x)).replace('x','x[0]').replace('y','x[1]') , str(f.diff(y)).replace('x','x[0]').replace('y','x[1]')
gradFunc()

('3*x[0]**2 + 3*x[1]**2 - 15', '6*x[0]*x[1] - 12')

In [63]:
def f(x):
    return x[0]**3 + 3*x[0]*x[1]**2 - 15*x[0] -12 * x[1]

# gradient
def f1(x):
    return np.array([3*x[0]**2 + 3*x[1]**2 - 15, 6*x[0]*x[1] - 12])

In [64]:
BFGS(f, f1)

Координаты точки экстремума : [1.99999751 0.99999747]
Значение функции в точке экстремума : -27.999999999886388
Точка, удовлетворяющая условиям найдена
время выполнения( в секундах ) = 0.0019958019256591797



(array([1.99999751, 0.99999747]), -27.999999999886388, 1)

### 5)

In [75]:
def func(var1 = 'x',var2 = 'y',f = '(x**2) * (y **2) + 0.5*x**2 + 0.5*y**2 + x*y + 1'):
    return f.replace(var1,'x[0]').replace(var2,'x[1]')
func()

'(x[0]**2) * (x[1] **2) + 0.5*x[0]**2 + 0.5*x[1]**2 + x[0]*x[1] + 1'

In [77]:
def gradFunc(var1 = 'x',var2 = 'y', f = '(x**2) * (y **2) + 0.5*x**2 + 0.5*y**2 + x*y + 1'):
    x = Symbol(var1)
    y = Symbol(var2)
    f = parse_expr(f)
    return str(f.diff(x)).replace('x','x[0]').replace('y','x[1]') , str(f.diff(y)).replace('x','x[0]').replace('y','x[1]')
gradFunc()

('2*x[0]*x[1]**2 + 1.0*x[0] + x[1]', '2*x[0]**2*x[1] + x[0] + 1.0*x[1]')

In [78]:
def f(x):
    return (x[0]**2) * (x[1] **2) + 0.5*x[0]**2 + 0.5*x[1]**2 + x[0]*x[1] + 1

# gradient
def f1(x):
    return np.array([2*x[0]*x[1]**2 + 1.0*x[0] + x[1], 2*x[0]**2*x[1] + x[0] + 1.0*x[1]])

In [80]:
BFGS(f, f1, x0 = [1,2])

Координаты точки экстремума : [-0.0076573   0.00762688]
Значение функции в точке экстремума : 1.0000000038733976
Точка, удовлетворяющая условиям найдена
время выполнения( в секундах ) = 0.0029914379119873047



(array([-0.0076573 ,  0.00762688]), 1.0000000038733976, 1)